In [16]:
import pickle
import json
import os
from glob import glob
from tqdm import tqdm
from datasets import Audio
from collections import defaultdict

sr = 16000
audio = Audio(sampling_rate=sr)

In [2]:
pkls = sorted(glob('/home/husein/ssd2/processed-youtube/*.pkl'))
len(pkls)

2541

In [3]:
with open('mapping-youtube-speakers-80.json') as fopen:
    unique_speakers = json.load(fopen)
    
len(unique_speakers)

9304

In [30]:
speakers = defaultdict(int)

for pkl in tqdm(pkls):
    with open(pkl, 'rb') as fopen:
        data = pickle.load(fopen)
        
    filename = os.path.split(pkl)[1].replace('.pkl', '')
        
    for result in data:
        speaker = result['classification_model'][1]
        y = audio.decode_example(audio.encode_example(result['wav_data']))['array']
        speaker_name = f'{filename}-{speaker}'
        if speaker_name not in unique_speakers:
            continue
        actual_speaker = unique_speakers[speaker_name]
        
        speakers[actual_speaker] += (len(y) / sr)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2541/2541 [04:56<00:00,  8.58it/s]


In [31]:
speakers

defaultdict(int,
            {'"Abam_tak_pernah_berkira,_minta_tolong_promo_tak_berbayar_pun_dia_buat"_-_Shuk_Sahar-nhLcpCpEPW4-speaker 0': 0.8881875,
             'Kasut_hujan___Episod_Penuh_Bing___Bing_Bahasa_Melayu-AFGRShwA_-k-speaker 0': 285966.7296874919,
             '"Jangan_sombong_jadi_orang"_-_Mobile_Legends_\'team_player\'_Indonesia-Q5GQS6na0ug-speaker 2': 0.8098125,
             '"Jangan_sombong_jadi_orang"_-_Mobile_Legends_\'team_player\'_Indonesia-Q5GQS6na0ug-speaker 3': 0.8881875,
             '"Jangan_sombong_jadi_orang"_-_Mobile_Legends_\'team_player\'_Indonesia-Q5GQS6na0ug-speaker 4': 1.33225,
             '"Jangan_sombong_jadi_orang"_-_Mobile_Legends_\'team_player\'_Indonesia-Q5GQS6na0ug-speaker 5': 0.8098125,
             '"Jangan_sombong_jadi_orang"_-_Mobile_Legends_\'team_player\'_Indonesia-Q5GQS6na0ug-speaker 6': 0.9665625,
             '"Saya_tak_minta_duit_orang,_cari_publisiti_murahan!"_-_Aliff_Syukri_terkesan_kecaman_netizen-7DKPiHNh7jw-speaker 1': 1.0971875,

In [32]:
with open('lengths-80.json', 'w') as fopen:
    json.dump(speakers, fopen)